In [1]:
import warnings
warnings.filterwarnings('ignore')
#from keras.utils import multi_gpu_model
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7798537734557392958
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4930941747
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11755431731168895332
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
import os
import tensorflow as tf
import glob
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
#import tensorflow.contrib.eager as tfe

from mode.annotation import *
from mode.box import *
from mode.darknet53 import *
from mode.generator import *
from mode.weights import *
from mode.yolohead import *
from mode.yolov3 import *
from mode.yololoss import *
import tensorflow as tf
from tensorflow.keras import backend as K
#from tensorflow.keras.backend import set_session 

In [3]:
#tf.debugging.set_log_device_placement(True)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [5]:
tf.test.is_gpu_available()
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [6]:

path='weight'
path_data='data'
#定义coco锚点候选框
COCO_ANCHORS = [10,13, 16,30, 33,23, 30,61, 62,45, 59,119, 116,90, 156,198, 373,326]
YOLOV3_WEIGHTS =os.path.join(path, "yolov3.weights")
LABELS = ["方標"]
imgsize =416
batch_size =32
ann_dir = os.path.join(path_data, "ann", "*.xml")
img_dir = os.path.join(path_data, "img")

#定义训练参数
learning_rate = 1e-5  #定义学习率
num_epoches =50   #定义迭代次数
save_dir = "./model"  #定义模型路径

yolo_v3= Yolonet(n_classes=len(LABELS))

In [7]:
YOLOV3_WEIGHTS

'weight\\yolov3.weights'

In [8]:
yolo_v3 = Yolonet(n_classes=len(LABELS))#实例化yolo模型类对象
yolo_v3.load_darknet_params(YOLOV3_WEIGHTS, skip_detect_layer=True)#加载预训练模型

(864,)
(18432,)
(2048,)
(18432,)
(73728,)
(8192,)
(73728,)
(8192,)
(73728,)
(294912,)
(32768,)
(294912,)
(32768,)
(294912,)
(32768,)
(294912,)
(32768,)
(294912,)
(32768,)
(294912,)
(32768,)
(294912,)
(32768,)
(294912,)
(32768,)
(294912,)
(1179648,)
(131072,)
(1179648,)
(131072,)
(1179648,)
(131072,)
(1179648,)
(131072,)
(1179648,)
(131072,)
(1179648,)
(131072,)
(1179648,)
(131072,)
(1179648,)
(131072,)
(1179648,)
(4718592,)
(524288,)
(4718592,)
(524288,)
(4718592,)
(524288,)
(4718592,)
(524288,)
(4718592,)
(524288,)
(4718592,)
(524288,)
(4718592,)
(524288,)
(4718592,)
(131072,)
(196608,)
(1179648,)
(131072,)
(1179648,)
(131072,)
(1179648,)
(32768,)
(49152,)
(294912,)
(32768,)
(294912,)
(32768,)
(294912,)
62001757


In [9]:
ann_dir

'data\\ann\\*.xml'

In [10]:
train_ann_fnames = glob.glob(ann_dir)

In [11]:
#制作数据集
generator = BatchGenerator(train_ann_fnames,img_dir,
                           net_size=imgsize,
                           anchors=COCO_ANCHORS,
                             batch_size=batch_size,
                             labels=LABELS,
                             jitter = False)#随机变化尺寸。数据增强

## 批次訓練32，解決gpu OOM的方法

## 我的gpu最高只能分割4

In [15]:
split_time=4

In [17]:
@tf.function
def loss_splt(yolo_v3,split_ys,data_x):
    split_x=yolo_v3(data_x)
    loss_value=0
    loss_value = loss_fn(split_ys, split_x,anchors=COCO_ANCHORS,image_size=[imgsize, imgsize])
    #loss_value /= generator.steps_per_epoch
    return loss_value


def _loop_validation(model, generator):
    n_steps = generator.steps_per_epoch
    loss_value = 0
    for _ in range(n_steps): #按批次循环获取数据，并计算loss
        xs, yolo_1, yolo_2, yolo_3 = generator.next_batch()#xs是圖片放大後歸一化
        xs=tf.convert_to_tensor(xs)
        yolo_1=tf.convert_to_tensor(yolo_1)
        yolo_2=tf.convert_to_tensor(yolo_2)
        yolo_3=tf.convert_to_tensor(yolo_3)        
        ys = [yolo_1, yolo_2, yolo_3]
        ys_ = model(xs )

        time=0
        for i in range(int(batch_size//split_time)):
            #data_x = [tf.expand_dims(ys_[0][i],0),tf.expand_dims(ys_[1][i],0),tf.expand_dims(ys_[2][i],0)]
            data_x = [ys_[0][time:split_time+time],ys_[1][time:split_time+time],ys_[2][time:split_time+time]]
            #split_ys=[tf.expand_dims(ys[0][i],0),tf.expand_dims(ys[1][i],0),tf.expand_dims(ys[2][i],0)]
            split_ys = [ys[0][time:split_time+time],ys[1][time:split_time+time],ys[2][time:split_time+time]]
            
            time+=split_time
            loss_value += loss_fn(split_ys, data_x,anchors=COCO_ANCHORS,image_size=[imgsize, imgsize])
    loss_value /= batch_size
    loss_value /= generator.steps_per_epoch
    return loss_value
    
#update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)  
@tf.function
def _loop_train(model,optimizer, generator):
    
    # one epoch
    n_steps = generator.steps_per_epoch
    for _ in tqdm(range(n_steps)):#按批次循环获取数据，并计算训练
        time=0
        xs, yolo_1, yolo_2, yolo_3 = generator.next_batch()
        xs=tf.convert_to_tensor(xs)
        yolo_1=tf.convert_to_tensor(yolo_1)
        yolo_2=tf.convert_to_tensor(yolo_2)
        yolo_3=tf.convert_to_tensor(yolo_3)
        ys = [yolo_1, yolo_2, yolo_3]

        for i in range(int(batch_size//split_time)):
            with tf.GradientTape() as tape:
                data_x = xs[time:split_time+time]  
                split_ys=[ys[0][time:split_time+time],ys[1][time:split_time+time],ys[2][time:split_time+time]]
                loss =loss_splt(model,split_ys,data_x)  
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients,  model.trainable_variables))

In [16]:
@tf.function
def train():
    n_steps = generator.steps_per_epoch
    for _ in tqdm(range(2)):#按批次循环获取数据，并计算训练
        time=0
        xs, yolo_1, yolo_2, yolo_3 = generator.next_batch()
        xs=tf.convert_to_tensor(xs)
        yolo_1=tf.convert_to_tensor(yolo_1)
        yolo_2=tf.convert_to_tensor(yolo_2)
        yolo_3=tf.convert_to_tensor(yolo_3)
        ys = [yolo_1, yolo_2, yolo_3]
        for i in range(int(batch_size//split_time)):
            with tf.GradientTape() as tape:
                data_x = xs[time:split_time+time]
                split_ys=[ys[0][time:split_time+time],ys[1][time:split_time+time],ys[2][time:split_time+time]]
                loss =loss_splt(yolo_v3,split_ys,data_x)  
            gradients = tape.gradient(loss, yolo_v3.trainable_variables)
            optimizer.apply_gradients(zip(gradients,  yolo_v3.trainable_variables))
            

In [17]:
def loss_splt(yolo_v3,split_ys,data_x):
    split_x=yolo_v3(data_x)
    loss_value=0
    loss_value = loss_fn(split_ys, split_x,anchors=COCO_ANCHORS,image_size=[imgsize, imgsize])
    #loss_value /= generator.steps_per_epoch
    return loss_value


def _loop_validation():
    start = timer()
    n_steps = generator.steps_per_epoch
    loss_value = 0
    for _ in range(2): #按批次循环获取数据，并计算loss
        xs, yolo_1, yolo_2, yolo_3 = generator.next_batch()#xs是圖片放大後歸一化
        xs=tf.convert_to_tensor(xs)
        yolo_1=tf.convert_to_tensor(yolo_1)
        yolo_2=tf.convert_to_tensor(yolo_2)
        yolo_3=tf.convert_to_tensor(yolo_3)        
        ys = [yolo_1, yolo_2, yolo_3]
        ys_ = yolo_v3(xs )

        time=0
        #for i in range(int(batch_size//1)):
            #data_x = [tf.expand_dims(ys_[0][i],0),tf.expand_dims(ys_[1][i],0),tf.expand_dims(ys_[2][i],0)]
            #data_x = [ys_[0][time:split_time+time],ys_[1][time:split_time+time],ys_[2][time:split_time+time]]
            #split_ys=[tf.expand_dims(ys[0][i],0),tf.expand_dims(ys[1][i],0),tf.expand_dims(ys[2][i],0)]
            #split_ys = [ys[0][time:split_time+time],ys[1][time:split_time+time],ys[2][time:split_time+time]]
            
            #time+=split_time
        loss_value += loss_fn(ys, ys_,anchors=COCO_ANCHORS,image_size=[imgsize, imgsize])
    #loss_value /= batch_size
    loss_value /= generator.steps_per_epoch
    end = timer()
    print(end - start)
    return loss_value

In [18]:
for i in range(2):
    start = timer()
    #_loop_train( yolo_v3,optimizer, generator)#训练
    train()
    end = timer()
    print(end - start)
    loss_value = _loop_validation()#验证
    print("{}-th loss = {}".format(i, loss_value))

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:26<00:00, 13.46s/it]


110.3401741
4.183177400000005
0-th loss = 2.9939568042755127
5.842236500000013
2.9902402000000166
1-th loss = 3.2516164779663086


In [19]:
yolo_v3.load_weights("model"+'\\weights'+".h5")

In [ ]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
save_fname = os.path.join(save_dir, "weights")
history = []
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
#grad = tfe.implicit_gradients(loss_splt)
for i in range(150):
    start = timer()
    #_loop_train( yolo_v3,optimizer, generator)#训练
    train()
    end = timer()
    print(end - start)

    loss_value = _loop_validation(yolo_v3, generator)#验证
    print("{}-th loss = {}".format(i, loss_value))

    #收集loss
    history.append(loss_value)
    if loss_value == min(history):#只有loss创新低时再保存模型
        print("    update weight {}".format(loss_value))
        yolo_v3.save_weights("{}.h5".format(save_fname))

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:30<00:00,  3.06s/it]


Executing op __inference_train_165506 in device /job:localhost/replica:0/task:0/device:GPU:0


# 批次訓練分割數1

In [26]:
def _loop_validation(model, generator):
    n_steps = generator.steps_per_epoch
    loss_value = 0
    for _ in range(n_steps): #按批次循环获取数据，并计算loss
        xs, yolo_1, yolo_2, yolo_3 = generator.next_batch()#xs是圖片放大後歸一化
        xs=tf.convert_to_tensor(xs)
        yolo_1=tf.convert_to_tensor(yolo_1)
        yolo_2=tf.convert_to_tensor(yolo_2)
        yolo_3=tf.convert_to_tensor(yolo_3)        
        ys = [yolo_1, yolo_2, yolo_3]
        ys_ = model(xs )

        time=0
        for i in range(int(batch_size//split_time)):
            #data_x = [tf.expand_dims(ys_[0][i],0),tf.expand_dims(ys_[1][i],0),tf.expand_dims(ys_[2][i],0)]
            data_x = [ys_[0][time:split_time+time],ys_[1][time:split_time+time],ys_[2][time:split_time+time]]
            #split_ys=[tf.expand_dims(ys[0][i],0),tf.expand_dims(ys[1][i],0),tf.expand_dims(ys[2][i],0)]
            split_ys = [ys[0][time:split_time+time],ys[1][time:split_time+time],ys[2][time:split_time+time]]
            
            time+=split_time
            loss_value += loss_fn(split_ys, data_x,anchors=COCO_ANCHORS,image_size=[imgsize, imgsize])
    loss_value /= batch_size
    loss_value /= generator.steps_per_epoch
    return loss_value

In [38]:
@tf.function
def _loop_validation():
    start = timer()
    n_steps = generator.steps_per_epoch
    loss_value = 0
    for _ in range(10): #按批次循环获取数据，并计算loss
        xs, yolo_1, yolo_2, yolo_3 = generator.next_batch()#xs是圖片放大後歸一化
        xs=tf.convert_to_tensor(xs)
        yolo_1=tf.convert_to_tensor(yolo_1)
        yolo_2=tf.convert_to_tensor(yolo_2)
        yolo_3=tf.convert_to_tensor(yolo_3)        
        ys = [yolo_1, yolo_2, yolo_3]
        ys_ = yolo_v3(xs )

        time=0
        #for i in range(int(batch_size//1)):
            #data_x = [tf.expand_dims(ys_[0][i],0),tf.expand_dims(ys_[1][i],0),tf.expand_dims(ys_[2][i],0)]
            #data_x = [ys_[0][time:split_time+time],ys_[1][time:split_time+time],ys_[2][time:split_time+time]]
            #split_ys=[tf.expand_dims(ys[0][i],0),tf.expand_dims(ys[1][i],0),tf.expand_dims(ys[2][i],0)]
            #split_ys = [ys[0][time:split_time+time],ys[1][time:split_time+time],ys[2][time:split_time+time]]
            
            #time+=split_time
        loss_value += loss_fn(ys, ys_,anchors=COCO_ANCHORS,image_size=[imgsize, imgsize])
    #loss_value /= batch_size
    loss_value /= generator.steps_per_epoch
    end = timer()
    print(end - start)
    return loss_value

In [14]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
save_fname = os.path.join(save_dir, "weights")
history = []
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
grad = tfe.implicit_gradients(loss_splt)
for i in range(150):
    with tf.device('/device:GPU:0'):
        start = timer()
        _loop_train( yolo_v3,optimizer, generator,grad)#训练
        end = timer()
        print(end - start)

        loss_value = _loop_validation(yolo_v3, generator)#验证
        print("{}-th loss = {}".format(i, loss_value))

    #收集loss
    history.append(loss_value)
    if loss_value == min(history):#只有loss创新低时再保存模型
        print("    update weight {}".format(loss_value))
        yolo_v3.save_weights("{}.h5".format(save_fname))

save_fname_ned='./model\\weights_end'
yolo_v3.save_weights("{}.h5".format(save_fname_ned))

100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [03:16<00:00,  4.56s/it]


196.0677308
0-th loss = 2.541799783706665
    update weight 2.541799783706665


100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [03:12<00:00,  4.47s/it]


192.32619320000003
1-th loss = 1.2254680395126343
    update weight 1.2254680395126343


100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [03:12<00:00,  4.47s/it]


...

191.6462812999962


  0%|                                                                                           | 0/43 [00:00<?, ?it/s]

148-th loss = 11867.8828125


100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [03:11<00:00,  4.46s/it]


191.70280900000216
149-th loss = 2001.6402587890625
